<a href="https://colab.research.google.com/github/nedokormysh/Stepik_Competitive_DS/blob/week_6/CompetitiveDS_6_4_1_optuna_cv_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
!pip install catboost -q
!pip install pytorch-tabnet -q
!pip install optuna -q
# !pip install lightgbm -q
# !pip install xgboost -q

# Загрузка библиотек

In [127]:
from typing import List
from typing import Any

import pandas as pd
import numpy as np

import optuna
from optuna.integration import CatBoostPruningCallback, LightGBMPruningCallback, XGBoostPruningCallback
optuna.logging.set_verbosity(optuna.logging.WARNING)

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, Dataset
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import ElasticNet
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import NuSVR

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import make_pipeline


from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
import copy


import pickle
import warnings
warnings.simplefilter('ignore')

import os
import platform
import psutil

In [128]:
# print(platform.system(), platform.processor())
# # print(os.uname())
# print('CPU count:', os.cpu_count())
# print('RAM (GB):', psutil.virtual_memory()[0]/1000000000)
# print('RAM memory % used:', psutil.virtual_memory()[2])
# !python -V

# Загрузка данных

In [129]:
PATH_TRAIN = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_train.csv'
PATH_TEST = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/quickstart_test.csv'

## Оптимизируем объём памяти.

In [130]:
def reduce_mem_usage(df: pd.DataFrame) -> pd.DataFrame:
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

def import_data(file: str)-> pd.DataFrame:
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [131]:
print('train')
train = import_data(PATH_TRAIN)

print('-' * 80)
print('test')
test = import_data(PATH_TEST)

train
Memory usage of dataframe is 0.30 MB
Memory usage after optimization is: 0.16 MB
Decreased by 47.7%
--------------------------------------------------------------------------------
test
Memory usage of dataframe is 0.23 MB
Memory usage after optimization is: 0.14 MB
Decreased by 40.9%


## Подготовка данных

In [132]:
cat_features = ['model', 'car_type', 'fuel_type']
cat_features_indxs = [0, 1, 2]
targets = [#'target_class',
           'target_reg']
features2drop = ['car_id', 'deviation_normal_count', 'target_class']

filtered_features = [i for i in train.columns if (i not in targets and i not in features2drop)]
num_features = [i for i in filtered_features if i not in cat_features]

print(f'\033[1mcat_features:\033[0m {cat_features}, \033[1mamount\033[0m = {len(cat_features)}')
print(f'\033[1mnum_features:\033[0m {num_features}, \033[1mamount\033[0m = {len(num_features)}')
print(f'\033[1mtargets:\033[0m {targets}, \033[1mamount\033[0m = {len(targets)}')

cat_features: ['model', 'car_type', 'fuel_type'], amount = 3
num_features: ['car_rating', 'year_to_start', 'riders', 'year_to_work', 'mean_rating', 'distance_sum', 'rating_min', 'speed_max', 'user_ride_quality_median', 'user_uniq'], amount = 10
targets: ['target_reg'], amount = 1


In [133]:
### Заменим тип данных object на category в категориальных признаках

for col in cat_features:
    train[col] = train[col].astype("category")

for col in cat_features:
    test[col] = test[col].astype("category")

# train.head()

In [134]:
# получаем разбиение на фичи и целевую переменную
X = train[filtered_features].drop(targets, axis=1, errors="ignore")
y = train["target_reg"]

In [135]:
# подготовка тестовых данных
X_sub = test[filtered_features].drop(targets, axis=1, errors="ignore")

In [136]:
# инициализируем случайное состояние, список лучших моделей, словарь для результатов
r_s = 7575
n_est = 400
bst_mdls = {}
bst_fold_scores = {}

df_sub = pd.DataFrame(test['car_id'])

In [137]:
# разбиение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=r_s)

## Вспомогательные функции и классы

### Класс для нормализации цифровых признаков

In [138]:
class CustomTransformer(TransformerMixin):
    '''
    Класс трансформации признаков для подачи в catboost.
    Числовые данные нормализуются или стандартизируются.
    Категориальные признаки не изменяются.
    На выходе получаем датафрейм.
    '''
    def __init__(self,
                 scaler_type: Any=None,
                 cat_feats: List[str]=None,
                 with_mean: bool=True,
                 with_std: bool=True,
                 feature_range: tuple[int, int]=(0, 1)):
        self.scaler_type = scaler_type
        if scaler_type:
            self.scaler = StandardScaler(with_mean=with_mean, with_std=with_std,)
        else:
            self.scaler = MinMaxScaler(feature_range=feature_range)

        self.cat_feats = cat_feats

        # print('Init done')
    def fit(self, X: pd.DataFrame, y: pd.Series=None):
        return self

    def transform(self, X: pd.DataFrame, y: pd.Series=None) -> pd.DataFrame:
        X_ = X.copy()
        X_cat = X_[self.cat_feats]
        num_feats = [i for i in X_.columns if i not in self.cat_feats]
        X_num = X_[num_feats]
        X_num = pd.DataFrame(self.scaler.fit_transform(X_num), columns=num_feats)

        res = pd.concat([X_cat.reset_index(), X_num], axis=1)
        return res.set_index('index')

### Класс для нормализации и преобразования для нейронной сети.

In [139]:
class CustomTransformerNN(TransformerMixin):
    '''
    Класс трансформации признаков для подачи в catboost.
    Числовые данные нормализуются или стандартизируются.
    Категориальные признаки не изменяются.
    На выходе получаем датафрейм.
    '''
    def __init__(self, scaler_type=None, cat_feats=None, with_mean=True, with_std=True, feature_range=(0, 1)):
        self.scaler_type = scaler_type
        if scaler_type:
            self.scaler = StandardScaler(with_mean=with_mean, with_std=with_std,)
        else:
            self.scaler = MinMaxScaler(feature_range=feature_range)

        self.cat_feats = cat_feats

        # print('Init done')
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None) -> np.array:
        X_ = X.copy()
        X_cat = X_[self.cat_feats]
        for col in cat_features:
            X_cat[col] = X_cat[col].replace(np.unique(X_cat[col]), np.arange(X_cat[col].nunique()))
        num_feats = [i for i in X_.columns if i not in self.cat_feats]
        X_num = X_[num_feats]
        X_num = pd.DataFrame(self.scaler.fit_transform(X_num), columns=num_feats)

        res = pd.concat([X_cat.reset_index(), X_num], axis=1)
        return res.set_index('index').values

### Функция для шага обучения

In [140]:
# функция для шага обучения
def lr_gen_simple(init_lr: float, epoch: int) -> List[float]:
    '''
    Функция для формирования шага обучения.

    Параметры.
    ----------
    init_lr: float - изначальный шаг
    epoch: int - количество эпох
    '''
    decay = init_lr / epoch
    lr_list = [init_lr/(1 + decay * i) for i in range(1, epoch + 1)]

    return lr_list

### Функция подготовки пайплайнов

In [141]:
def preparing_features(name, algorithm):

    if name == 'CatBoostRegressor':
          preprocessor = CustomTransformer(True, cat_feats=cat_features)

          pipe = Pipeline([('feature_preprocessor', preprocessor),
                           ('model', algorithm)])
    elif name == 'TabNetRegressor':
        preprocessor = CustomTransformerNN(True, cat_feats=cat_features)

        pipe = Pipeline([('feature_preprocessor', preprocessor),
                          ('model', algorithm)])
    # elif name == 'LGBMRegressor':
    else:
        numeric_transformer = Pipeline(
            steps=[("imputer", SimpleImputer(strategy="median")),
                   ("scaler", StandardScaler())])

        categorical_transformer = Pipeline(
            steps=[("encoder", OneHotEncoder(handle_unknown="ignore")),
                  #  ("selector", SelectPercentile(chi2, percentile=50)),
                   ])
        preprocessor = ColumnTransformer(
            transformers=[("num", numeric_transformer, num_features),
                          ("cat", categorical_transformer, cat_features)])

        pipe = Pipeline([('feature_preprocessor', preprocessor),
                         ('model', algorithm)])


    return preprocessor, pipe

### Best Fold

In [142]:
# функция для получения модели лучшего фолда
def best_fold_model(algorithm, X, y, params=None, fit_params=None, n_splits=3,
                    r_s=7575, n_rounds=1000, stop_rounds=200, init_lr=0.12, callbacks=[]):
    regrs = []
    RMSE_scores = []

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    for num, (train_index, val_index) in enumerate(kf.split(X)):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        if params != None:
            rgr = algorithm(**params)
        else:
            # regr = algorithm()
            print(f'No params')
            pass

        preprocessor, pipe = preparing_features(algorithm.__name__, rgr)

        target_sc = None

        # print(callbacks)


        if algorithm.__name__ == 'TabNetRegressor':
            target_sc = StandardScaler()
            y_train = target_sc.fit_transform(y_train.values.reshape(-1, 1))
            X_val_eval_NN = X_val.copy()
            eval_set_pipe = Pipeline(steps = [('feature_preprocessor', preprocessor)])
            X_val_eval_NN = eval_set_pipe.fit(X_train).transform(X_val_eval_NN)
            y_val_eval_NN = target_sc.transform(y_val.values.reshape(-1, 1))
        else:
            X_val_eval = X_val.copy()
            # Создадим pipeline препроцессора без модели
            eval_set_pipe = Pipeline(steps = [('feature_preprocessor', preprocessor)])
            X_val_eval = eval_set_pipe.fit(X_train).transform(X_val_eval)

        # print(X_train.shape, X_val_eval.shape)

        wrapped_model = TransformedTargetRegressor(regressor=pipe, transformer=StandardScaler())
        # print(wrapped_model)

        if algorithm.__name__ in ['KNeighborsRegressor', 'ElasticNet', 'RandomForestRegressor', 'GaussianProcessRegressor', 'NuSVR', 'MLPRegressor']:
            wrapped_model.fit(X_train,
                           y_train,
                           **fit_params
                           )
            y_pred = wrapped_model.predict(X_val)
        elif algorithm.__name__ == 'TabNetRegressor':
            wrapped_model = pipe
            wrapped_model.fit(X_train, y_train,
                 model__eval_set=[(X_val_eval_NN, y_val_eval_NN)],
                 model__eval_metric = ['rmse']
            )
            y_pred = wrapped_model.predict(X_val)
            y_pred = target_sc.inverse_transform(np.array(y_pred).reshape(-1, 1))

        else:
            if callbacks:
                wrapped_model.fit(X_train,
                                  y_train,
                                  **fit_params,
                                  model__callbacks=callbacks,
                                  model__eval_set=[(X_val_eval, y_val)],
                          )
            else:
                wrapped_model.fit(X_train,
                          y_train,
                          # model__callbacks=callbacks,
                          model__eval_set=[(X_val_eval, y_val)],
                          **fit_params
                          )

            y_pred = wrapped_model.predict(X_val)

        regrs.append(wrapped_model)


        # выводим номер фолда и значение RMSE
        RMSE = mean_squared_error(y_true=y_val.values, y_pred=y_pred, squared=False)
        print(f'fold: {num}, test_RMSE: {RMSE}')
        # выводим номер лучшего фолда
        RMSE_scores.append(RMSE)
        min_RMSE = min(RMSE_scores)
        bst = regrs[np.argmin(RMSE_scores)]
        print(f'Best fold number = {np.argmin(RMSE_scores)}\n')

    assert len(regrs) == n_splits

    print('-' * 80)
    print(f'Среднее значение метрики по {n_splits} фолдам \033[1mtest_RMSE_mean\033[0m = {np.mean(RMSE_scores).round(4)}')

    if target_sc:
        return bst, algorithm.__name__, min_RMSE, np.mean(RMSE_scores).round(4), np.std(RMSE_scores).round(4), target_sc
    else:
        return bst, algorithm.__name__, min_RMSE, np.mean(RMSE_scores).round(4), np.std(RMSE_scores).round(4)

### Fit algorithm

In [143]:
def fit_algorithm(trial,
                  train: pd.DataFrame,
                  val: pd.DataFrame,
                  algorithm,
                  init_params,
                  grid,
                  fit_params,
                  r_s=r_s, callbacks = []):

    X_train, y_train = train
    X_val, y_val = val



    if algorithm.__name__ == 'CatBoostRegressor':
        if grid["bootstrap_type"] == "Bayesian":
            grid["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 20)

        elif grid["bootstrap_type"] == "Bernoulli":
            grid["subsample"] = trial.suggest_float("subsample", 0.1, 1)
    elif algorithm.__name__ == 'XGBRegressor':
        if grid["booster"] == "gbtree" or grid["booster"] == "dart":
            grid["max_depth"] = trial.suggest_int("max_depth", 1, 9)
            grid["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
            grid["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            grid["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
            grid["tree_method"]: trial.suggest_categorical("tree_method", ['hist', 'approx'],)
        if grid["booster"] == "dart":
            grid["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
            grid["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
            grid["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
            grid["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)


    rgr = algorithm(**init_params, **grid)

    preprocessor, pipe = preparing_features(algorithm.__name__, rgr)

    # print(callbacks)


    # if algorithm.__name__ == 'XGBRegressor':
    #     xgb.set_config(verbosity=0)

    if algorithm.__name__ == 'TabNetRegressor':
        target_sc = StandardScaler()
        y_train = target_sc.fit_transform(y_train.values.reshape(-1, 1))
        X_val_eval_NN = X_val.copy()
        eval_set_pipe = Pipeline(steps = [('feature_preprocessor', preprocessor)])
        X_val_eval_NN = eval_set_pipe.fit(X_train).transform(X_val_eval_NN)
        y_val_eval_NN = target_sc.transform(y_val.values.reshape(-1, 1))
    else:
        X_val_eval = X_val.copy()
        eval_set_pipe = Pipeline(steps = [('feature_preprocessor', preprocessor)])
        X_val_eval = eval_set_pipe.fit(X_train).transform(X_val_eval)


    wrapped_model = TransformedTargetRegressor(regressor=pipe, transformer=StandardScaler())

    if algorithm.__name__ in ['KNeighborsRegressor', 'ElasticNet', 'RandomForestRegressor', 'GaussianProcessRegressor', 'NuSVR', 'MLPRegressor']:
         wrapped_model.fit(X_train,
                           y_train,
                           **fit_params
                           )
         y_pred = wrapped_model.predict(X_val)
    elif algorithm.__name__ == 'TabNetRegressor':
        pipe.fit(X_train, y_train,
                 model__eval_set=[(X_val_eval_NN, y_val_eval_NN)],
                #  callbacks,
                 **fit_params,
                 )
        y_pred = pipe.predict(X_val)
        y_pred = target_sc.inverse_transform(np.array(y_pred).reshape(-1, 1))
    else:
        if callbacks:
            wrapped_model.fit(X_train,
                          y_train,
                          **fit_params,
                          model__callbacks=callbacks,
                          model__eval_set=[(X_val_eval, y_val)],
                          )
            # # # # запускаем процесс прунинга для catboost
            if algorithm.__name__ == 'CatBoostRegressor':
                 callbacks[-1].check_pruned()
        else:
            wrapped_model.fit(X_train,
                              y_train,
                              model__eval_set=[(X_val_eval, y_val)],
                              **fit_params)

        y_pred = wrapped_model.predict(X_val)

    return wrapped_model, y_pred

# Модели

## Catboost

In [144]:
def objective_catboost(trial, return_models=False):

    pruning_callback = CatBoostPruningCallback(trial, "RMSE")
    callbacks = [pruning_callback]

    init_params = {'loss_function': 'RMSE',
                   'random_seed': r_s,
                   'thread_count': -1,
                   'verbose': 0,
                   'task_type': "CPU",
                   'iterations' : n_est, # Можно не перебирать, есть Early-Stopping,
                   'cat_features': cat_features,
    }

    grid = {'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.01),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 100, 500),
            'l2_leaf_reg': trial.suggest_int("l2_leaf_reg", 2, 50),
            'max_bin': trial.suggest_int('max_bin', 100, 300),
            'boosting_type': trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
            'bootstrap_type': trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
            'depth': trial.suggest_int("depth", 3, 10),
            'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.01, 0.8),
            # 'subsample': trial.suggest_float("colsample_bylevel", 0.01, 0.95),
            'used_ram_limit': "14gb"
        }



    fit_params = {'model__cat_features': cat_features,
                  'model__verbose': 0,
                  'model__early_stopping_rounds': 200,
                  'model__plot': False
                  }

    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    scores, models = [], []

    for train_idx, valid_idx in kf.split(X):
        train_data = X.iloc[train_idx, :], y.iloc[train_idx]
        valid_data = X.iloc[valid_idx, :], y.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_algorithm(trial,
                                      train_data,
                                      valid_data,
                                      CatBoostRegressor,
                                      init_params,
                                      grid,
                                      fit_params,
                                      callbacks=callbacks)
        y_true = valid_data[1]
        scores.append(mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
        models.append(model)
        break

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [145]:
%%time
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            #  pruner=optuna.pruners.HyperbandPruner(),
                            direction="minimize", study_name="Catboost_Regressor")
study.optimize(objective_catboost,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True,)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 5min 5s, sys: 13.6 s, total: 5min 19s
Wall time: 3min 29s


In [146]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 12.03955263254623, params {'learning_rate': 0.009216774275725322, 'min_data_in_leaf': 316, 'l2_leaf_reg': 2, 'max_bin': 113, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'depth': 9, 'colsample_bylevel': 0.7005870831433161}


In [147]:
init_params = {'loss_function': 'RMSE',
                'random_seed': r_s,
                'thread_count': -1,
                'verbose': 0,
                'task_type': "CPU",
                'iterations' : n_est, # Можно не перебирать, есть Early-Stopping,
                'cat_features': cat_features,}

fit_params = {'model__cat_features': cat_features,
              'model__verbose': 0,
              'model__early_stopping_rounds': 200,
              'model__plot': False}

best_catboost_params = init_params | study.best_trial.params


res = best_fold_model(CatBoostRegressor, X, y, best_catboost_params, fit_params)

fold: 0, test_RMSE: 12.03955263254623
Best fold number = 0

fold: 1, test_RMSE: 11.873983451119146
Best fold number = 1

fold: 2, test_RMSE: 11.424122917098806
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по 3 фолдам test_RMSE_mean = 11.7792


In [148]:
bst_mdls.update({'Catboost_model': res[0]})
bst_fold_scores.update({'Catboost_target': res[2]})
bst_fold_scores, bst_mdls

({'Catboost_target': 11.424122917098806},
 {'Catboost_model': TransformedTargetRegressor(regressor=Pipeline(steps=[('feature_preprocessor',
                                                        <__main__.CustomTransformer object at 0x79e54b83f640>),
                                                       ('model',
                                                        <catboost.core.CatBoostRegressor object at 0x79e53ad9b0d0>)]),
                             transformer=StandardScaler())})

In [149]:
preds = res[0].predict(X_sub)

df_sub['Catboost_target'] = preds
df_sub.head()

,car_id,Catboost_target
0,P17494612l,46.076274
1,N-1530212S,33.787246
2,B-1154399t,33.687965
3,F12725233R,34.721955
4,l-1139189J,33.800808


## LGBM

In [150]:
def objective_lgbm(trial, return_models=False):

    pruning_callback = LightGBMPruningCallback(trial, "rmse")

    callbacks = [
                 lgb.early_stopping(stopping_rounds=50, verbose=0),
                 lgb.reset_parameter(learning_rate=lr_gen_simple(trial.suggest_float("lr_init", 0.0001, 0.5), n_est)),
                 pruning_callback
                ]

    init_params = {'metric': "RMSE",
                   'objective': 'regression',
                   'boosting_type': 'goss',
                   'random_seed': r_s,
                   'n_jobs': -1,
                   'verbose': 0,
                   'verbose_eval': -1,
                   'force_row_wise': True,
                   'num_iterations' : n_est, # Можно не перебирать, есть Early-Stopping
                  #  'cat_features': cat_features, # в данной реализации функции нам требуется подавать индексы
    }

    grid = {
            #  'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.01),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 50, 100),
            'min_child_samples': trial.suggest_int('min_child_samples', 100, 500),
            'reg_lambda': trial.suggest_int("reg_lambda", 2, 50),
            'max_bin': trial.suggest_int('max_bin', 100, 300),
            'num_leaves': trial.suggest_int('num_leaves', 2, 10),
            'colsample_bytree': trial.suggest_float("colsample_bytree", 0.01, 0.95),
            'subsample': trial.suggest_float("subsample", 0.01, 0.95),
        }

    fit_params = {}

    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    scores, models = [], []


    for train_idx, valid_idx in kf.split(X):
        train_data = X.iloc[train_idx, :], y.iloc[train_idx]
        valid_data = X.iloc[valid_idx, :], y.iloc[valid_idx]


        # Подаем trials для перебора
        model, y_pred = fit_algorithm(trial,
                                      train_data,
                                      valid_data,
                                      LGBMRegressor,
                                      init_params,
                                      grid,
                                      fit_params,
                                      callbacks=callbacks)

        y_true = valid_data[1]
        scores.append(mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
        models.append(model)
        break

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [151]:
%%time
# optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            # pruner=optuna.pruners.HyperbandPruner(#n_warmup_steps=5
                            #                                ),

                            direction="minimize", study_name="LGBM_Regressor")
study.optimize(objective_lgbm,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True,)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 17.9 s, sys: 1.5 s, total: 19.5 s
Wall time: 21.9 s


In [152]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 12.227809411913098, params {'lr_init': 0.11853268479306604, 'min_data_in_leaf': 58, 'min_child_samples': 133, 'reg_lambda': 20, 'max_bin': 101, 'num_leaves': 5, 'colsample_bytree': 0.8527745413986161, 'subsample': 0.6143584292597737}


In [153]:
best_LGBM_params = study.best_trial.params

lr_init = best_LGBM_params.pop('lr_init')


init_params = {'metric': "RMSE",
               'objective': 'regression',
                # 'boosting_type': 'goss',
               'random_seed': r_s,
               'n_jobs': -1,
               'verbose': -1,
              #  'verbose_eval': -1,
               'verbosity': -1,
               'force_row_wise': True,
               'num_iterations' : n_est, # Можно не перебирать, есть Early-Stopping
               #  'cat_features': cat_features, # в данной реализации функции нам требуется подавать индексы
    }
fit_params = {#'model__verbose': -1
              # 'model__silent': True,
              }

callbacks = [lgb.early_stopping(stopping_rounds=50, verbose=-100),
                         lgb.reset_parameter(learning_rate=lr_gen_simple(lr_init, n_est)),
                        #  lgb.log_evaluation(period=0)
                         ]

best_LGBM_params = init_params | best_LGBM_params

res = best_fold_model(LGBMRegressor, X, y, best_LGBM_params, fit_params, callbacks=callbacks)

[LightGBM] [Info] Start training from score -0.000030
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[396]	valid_0's rmse: 49.2357
fold: 0, test_RMSE: 12.142560359985131
Best fold number = 0

[LightGBM] [Info] Start training from score -0.000002
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[204]	valid_0's rmse: 47.7827
fold: 1, test_RMSE: 12.115951919881805
Best fold number = 1

[LightGBM] [Info] Start training from score -0.000034
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[133]	valid_0's rmse: 47.2228
fold: 2, test_RMSE: 11.566209261852029
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по 3 фолдам test_RMSE_mean = 11.9416


In [154]:
bst_mdls.update({'LGBM_model': res[0]})
bst_fold_scores.update({'LGBM_target': res[2]})
bst_fold_scores, bst_mdls

({'Catboost_target': 11.424122917098806, 'LGBM_target': 11.566209261852029},
 {'Catboost_model': TransformedTargetRegressor(regressor=Pipeline(steps=[('feature_preprocessor',
                                                        <__main__.CustomTransformer object at 0x79e54b83f640>),
                                                       ('model',
                                                        <catboost.core.CatBoostRegressor object at 0x79e53ad9b0d0>)]),
                             transformer=StandardScaler()),
  'LGBM_model': TransformedTargetRegressor(regressor=Pipeline(steps=[('feature_preprocessor',
                                                        ColumnTransformer(transformers=[('num',
                                                                                         Pipeline(steps=[('imputer',
                                                                                                          SimpleImputer(strategy='median')),
                     

In [155]:
preds = res[0].predict(X_sub)

df_sub['LGBM_target'] = preds
df_sub.head()

,car_id,Catboost_target,LGBM_target
0,P17494612l,46.076274,50.694532
1,N-1530212S,33.787246,33.181376
2,B-1154399t,33.687965,30.189062
3,F12725233R,34.721955,36.430008
4,l-1139189J,33.800808,30.863509


## TabNet

In [156]:
def objective_tabnet(trial, return_models=False):

    init_params = {'device_name':'cpu',
                   'cat_idxs': cat_features_indxs,
                   'cat_dims': [26, 4, 2],
                   'verbose': 0,
                  #  'verbose_eval': 0,
                   'seed': r_s,
                   'optimizer_fn': torch.optim.Adam,
                   'optimizer_params': dict(lr=2e-2, weight_decay=1e-5),
                   'scheduler_fn': torch.optim.lr_scheduler.ReduceLROnPlateau,

    }

    grid = {
             'mask_type': trial.suggest_categorical("mask_type", ["entmax", "sparsemax"]),
             'n_d': trial.suggest_int("n_d", 2, 20, step=4),
             'n_a': trial.suggest_int("n_a", 2, 20, step=4),
             'n_steps': trial.suggest_int("n_steps", 1, 3, step=1),
             'gamma': trial.suggest_float("gamma", 1., 1.4, step=0.2),
             'cat_emb_dim': trial.suggest_int("cat_emb_dim", 1, 10, step=2),
             'n_independent': trial.suggest_int("n_independent", 1, 3),
             'n_shared': trial.suggest_int("n_shared", 1, 3),
             'momentum': trial.suggest_float("momentum", .01, .4, step=0.2),
             'lambda_sparse': trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True),
             'scheduler_params': dict(mode="min",
                                      patience=trial.suggest_int("patienceScheduler", low=3, high=10), # changing sheduler patience to be lower than early stopping patience
                                      min_lr=1e-5,
                                      factor=0.5,
                                      # verbose=-1
                                      )
    }

    fit_params = {#'model__verbose: 0,
                  #'model__eval_verbose': 0,
                  'model__eval_metric':['rmse'],
                  'model__patience': trial.suggest_int("model_patience", low=15, high=30),
                  'model__max_epochs': 100
                  }

    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    scores, models = [], []


    for train_idx, valid_idx in kf.split(X):
        train_data = X.iloc[train_idx, :], y.iloc[train_idx]
        valid_data = X.iloc[valid_idx, :], y.iloc[valid_idx]


        # Подаем trials для перебора
        model, y_pred = fit_algorithm(trial,
                                      train_data,
                                      valid_data,
                                      TabNetRegressor,
                                      init_params,
                                      grid,
                                      fit_params)

        y_true = valid_data[1]
        scores.append(mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
        models.append(model)
        break

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [157]:
%%time
# optuna.logging.set_verbosity(optuna.logging.WARNING)
# optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(#pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            direction="minimize", study_name="TabNet_Regressor")
study.optimize(objective_tabnet,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True,)

  0%|          | 0/100 [00:00<?, ?it/s]


Early stopping occurred at epoch 97 with best_epoch = 68 and best_val_0_rmse = 0.7470899820327759
Stop training because you reached max_epochs = 100 with best_epoch = 99 and best_val_0_rmse = 0.773829996585846

Early stopping occurred at epoch 91 with best_epoch = 65 and best_val_0_rmse = 0.7507799863815308
Stop training because you reached max_epochs = 100 with best_epoch = 99 and best_val_0_rmse = 0.8527899980545044

Early stopping occurred at epoch 95 with best_epoch = 77 and best_val_0_rmse = 0.8704000115394592

Early stopping occurred at epoch 76 with best_epoch = 61 and best_val_0_rmse = 0.7509499788284302

Early stopping occurred at epoch 81 with best_epoch = 54 and best_val_0_rmse = 0.7439500093460083

Early stopping occurred at epoch 83 with best_epoch = 65 and best_val_0_rmse = 0.699180006980896
Stop training because you reached max_epochs = 100 with best_epoch = 93 and best_val_0_rmse = 0.7415000200271606
Stop training because you reached max_epochs = 100 with best_epoch = 

In [158]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 12.1088285446167, params {'mask_type': 'entmax', 'n_d': 10, 'n_a': 14, 'n_steps': 3, 'gamma': 1.2, 'cat_emb_dim': 5, 'n_independent': 2, 'n_shared': 2, 'momentum': 0.01, 'lambda_sparse': 0.00012285467222694234, 'patienceScheduler': 8, 'model_patience': 20}


In [159]:
best_params_NN = study.best_trial.params
patience_scheduler =  best_params_NN.pop('patienceScheduler')
patience_model = best_params_NN.pop('model_patience')

# patienceScheduler_int


init_params = {'device_name':'cpu',
               'cat_idxs': cat_features_indxs,
               'cat_dims': [26, 4, 2],
               'verbose': 0,
               'seed': 7575,
               'optimizer_fn': torch.optim.Adam,
               'optimizer_params': dict(lr=2e-2, weight_decay=1e-5),
               'scheduler_fn': torch.optim.lr_scheduler.ReduceLROnPlateau,
               'scheduler_params': dict(mode="min",
                                            patience=patience_scheduler,
                                            min_lr=1e-5,
                                            factor=0.5,)
    }

fit_params = {#'model__eval_verbose': 0,
              'model__eval_metric':['rmse'],
              'model__patience': patience_model,
              'model__max_epochs': 100}

best_TabNet_params = init_params | best_params_NN

res = best_fold_model(TabNetRegressor, X, y, best_TabNet_params, fit_params)


Early stopping occurred at epoch 60 with best_epoch = 50 and best_val_0_rmse = 0.7272999882698059
fold: 0, test_RMSE: 12.664419174194336
Best fold number = 0


Early stopping occurred at epoch 76 with best_epoch = 66 and best_val_0_rmse = 0.728190004825592
fold: 1, test_RMSE: 12.810641288757324
Best fold number = 0


Early stopping occurred at epoch 73 with best_epoch = 63 and best_val_0_rmse = 0.6888499855995178
fold: 2, test_RMSE: 12.334203720092773
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по 3 фолдам test_RMSE_mean = 12.603099822998047


In [160]:
bst_mdls.update({'TabNet_model': res[0]})
bst_fold_scores.update({'TabNet_target': res[2]})
bst_fold_scores, bst_mdls

({'Catboost_target': 11.424122917098806,
  'LGBM_target': 11.566209261852029,
  'TabNet_target': 12.334204},
 {'Catboost_model': TransformedTargetRegressor(regressor=Pipeline(steps=[('feature_preprocessor',
                                                        <__main__.CustomTransformer object at 0x79e54b83f640>),
                                                       ('model',
                                                        <catboost.core.CatBoostRegressor object at 0x79e53ad9b0d0>)]),
                             transformer=StandardScaler()),
  'LGBM_model': TransformedTargetRegressor(regressor=Pipeline(steps=[('feature_preprocessor',
                                                        ColumnTransformer(transformers=[('num',
                                                                                         Pipeline(steps=[('imputer',
                                                                                                          SimpleImputer(strategy='

In [161]:
target_sc = res[5]
preds = res[0].predict(X_sub)

preds = target_sc.inverse_transform(np.array(preds).reshape(-1, 1))

df_sub['TabNet_target'] = preds
df_sub.head()

,car_id,Catboost_target,LGBM_target,TabNet_target
0,P17494612l,46.076274,50.694532,34.337772
1,N-1530212S,33.787246,33.181376,33.622017
2,B-1154399t,33.687965,30.189062,32.035740
3,F12725233R,34.721955,36.430008,35.568596
4,l-1139189J,33.800808,30.863509,32.248013


## KNN regressor

In [162]:
def objective_knn(trial, return_models=False):
    init_params = { 'n_jobs': -1 }

    grid = {
             'weights': trial.suggest_categorical("weights", ['uniform', 'distance'],),
             'n_neighbors': trial.suggest_int("n_neighbors", 150, 250, step=4),
             'leaf_size': trial.suggest_int("leaf_size", 150, 250, step=4),
             'p':  trial.suggest_int("p", 1, 2)
             }

    fit_params = {}

    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    scores, models = [], []


    for train_idx, valid_idx in kf.split(X):
        train_data = X.iloc[train_idx, :], y.iloc[train_idx]
        valid_data = X.iloc[valid_idx, :], y.iloc[valid_idx]


        # Подаем trials для перебора
        model, y_pred = fit_algorithm(trial,
                                      train_data,
                                      valid_data,
                                      KNeighborsRegressor,
                                      init_params,
                                      grid,
                                      fit_params)

        y_true = valid_data[1]
        scores.append(mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
        models.append(model)
        break

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [163]:
%%time
# optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize", study_name="KNN_Regressor")
study.optimize(objective_knn,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True,)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 18.2 s, sys: 330 ms, total: 18.5 s
Wall time: 13.7 s


In [164]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 15.550230327421838, params {'weights': 'distance', 'n_neighbors': 150, 'leaf_size': 190, 'p': 2}


In [165]:
init_params = { 'n_jobs': -1 }
fit_params = { }

best_KNN_params = init_params | study.best_trial.params

res = best_fold_model(KNeighborsRegressor, X, y, best_KNN_params, fit_params)

fold: 0, test_RMSE: 15.550230327421838
Best fold number = 0

fold: 1, test_RMSE: 15.167035413589185
Best fold number = 1

fold: 2, test_RMSE: 14.701417459142696
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по 3 фолдам test_RMSE_mean = 15.1396


In [166]:
bst_mdls.update({'KNN_model': res[0]})
bst_fold_scores.update({'KNN_target': res[2]})
bst_fold_scores

preds = res[0].predict(X_sub)

df_sub['KNN_target'] = preds
df_sub.head()

,car_id,Catboost_target,LGBM_target,TabNet_target,KNN_target
0,P17494612l,46.076274,50.694532,34.337772,42.158554
1,N-1530212S,33.787246,33.181376,33.622017,43.264945
2,B-1154399t,33.687965,30.189062,32.035740,37.782325
3,F12725233R,34.721955,36.430008,35.568596,43.918047
4,l-1139189J,33.800808,30.863509,32.248013,47.391793


## Elastic net regressor

In [167]:
def objective_elastik(trial, return_models=False):
    init_params = {'random_state': r_s,}

    grid = {
             'max_iter': trial.suggest_int("max_iter", 500, 1000, step=4),
             'l1_ratio': trial.suggest_float("l1_ratio", 0.001, 1),
             'alpha':  trial.suggest_float("alpha", 0.001, 1),
             }

    fit_params = {}

    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    scores, models = [], []


    for train_idx, valid_idx in kf.split(X):
        train_data = X.iloc[train_idx, :], y.iloc[train_idx]
        valid_data = X.iloc[valid_idx, :], y.iloc[valid_idx]


        # Подаем trials для перебора
        model, y_pred = fit_algorithm(trial,
                                      train_data,
                                      valid_data,
                                      ElasticNet,
                                      init_params,
                                      grid,
                                      fit_params)

        y_true = valid_data[1]
        scores.append(mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
        models.append(model)
        break

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [168]:
%%time
# optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize", study_name="Elastik_Regressor")
study.optimize(objective_elastik,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True,)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 8.21 s, sys: 261 ms, total: 8.47 s
Wall time: 8.44 s


In [169]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 14.819536861766776, params {'max_iter': 832, 'l1_ratio': 0.11725927492297664, 'alpha': 0.0010719573502393884}


In [170]:
init_params = { 'random_state': r_s }
fit_params = { }

best_Elastik_params = init_params | study.best_trial.params

res = best_fold_model(ElasticNet, X, y, best_Elastik_params, fit_params)

fold: 0, test_RMSE: 14.819536861766776
Best fold number = 0

fold: 1, test_RMSE: 14.439893325533072
Best fold number = 1

fold: 2, test_RMSE: 14.448504138343624
Best fold number = 1

--------------------------------------------------------------------------------
Среднее значение метрики по 3 фолдам test_RMSE_mean = 14.5693


In [171]:
bst_mdls.update({'Elastic_model': res[0]})
bst_fold_scores.update({'Elastic_target': res[2]})
bst_fold_scores

preds = res[0].predict(X_sub)

df_sub['Elastic_target'] = preds
df_sub.head()

,car_id,Catboost_target,LGBM_target,TabNet_target,KNN_target,Elastic_target
0,P17494612l,46.076274,50.694532,34.337772,42.158554,48.730902
1,N-1530212S,33.787246,33.181376,33.622017,43.264945,43.930278
2,B-1154399t,33.687965,30.189062,32.035740,37.782325,38.445516
3,F12725233R,34.721955,36.430008,35.568596,43.918047,37.973594
4,l-1139189J,33.800808,30.863509,32.248013,47.391793,51.561601


## Random Forrest regressor

In [172]:
def objective_random_forest(trial, return_models=False):
    init_params = {'random_state': r_s,
                   'n_jobs': -1}

    grid = { 'n_estimators': trial.suggest_int("n_estimators", 500, 1000, step=4),
             'max_depth': trial.suggest_int("max_depth", 2, 12),
             'max_samples': trial.suggest_int("max_samples", 200, 400),
             'min_samples_leaf': trial.suggest_float("min_samples_leaf", 0.5, 1),
             }

    fit_params = {}

    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    scores, models = [], []


    for train_idx, valid_idx in kf.split(X):
        train_data = X.iloc[train_idx, :], y.iloc[train_idx]
        valid_data = X.iloc[valid_idx, :], y.iloc[valid_idx]


        # Подаем trials для перебора
        model, y_pred = fit_algorithm(trial,
                                      train_data,
                                      valid_data,
                                      RandomForestRegressor,
                                      init_params,
                                      grid,
                                      fit_params)

        y_true = valid_data[1]
        scores.append(mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
        models.append(model)
        break

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [173]:
%%time
# optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize", study_name="Random_Forest_Regressor")
study.optimize(objective_random_forest,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True,)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 1min 59s, sys: 12 s, total: 2min 11s
Wall time: 2min 9s


In [174]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 18.118155925922782, params {'n_estimators': 536, 'max_depth': 10, 'max_samples': 240, 'min_samples_leaf': 0.8329510466033764}


In [175]:
%%time

init_params = {'random_state': r_s,
               'n_jobs': -1}

fit_params = { }

best_Random_forest_params = init_params | study.best_trial.params

res = best_fold_model(RandomForestRegressor, X, y, best_Random_forest_params, fit_params)

fold: 0, test_RMSE: 18.118155925922782
Best fold number = 0

fold: 1, test_RMSE: 17.750203624482527
Best fold number = 1

fold: 2, test_RMSE: 17.108867104504995
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по 3 фолдам test_RMSE_mean = 17.6591
CPU times: user 3.11 s, sys: 220 ms, total: 3.33 s
Wall time: 3.34 s


In [176]:
bst_mdls.update({'RF_model': res[0]})
bst_fold_scores.update({'RF_target': res[2]})
bst_fold_scores

preds = res[0].predict(X_sub)

df_sub['RF_target'] = preds
df_sub.head()

,car_id,Catboost_target,LGBM_target,TabNet_target,KNN_target,Elastic_target,RF_target
0,P17494612l,46.076274,50.694532,34.337772,42.158554,48.730902,45.247314
1,N-1530212S,33.787246,33.181376,33.622017,43.264945,43.930278,45.247314
2,B-1154399t,33.687965,30.189062,32.035740,37.782325,38.445516,45.247314
3,F12725233R,34.721955,36.430008,35.568596,43.918047,37.973594,45.247314
4,l-1139189J,33.800808,30.863509,32.248013,47.391793,51.561601,45.247314


## Gaussian Process regressor

In [177]:
def objective_gaussian(trial, return_models=False):
    init_params = {'random_state': r_s,}

    grid = {
            'alpha': trial.suggest_float("alpha", 0.2, 1),
           }

    fit_params = {}

    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    scores, models = [], []


    for train_idx, valid_idx in kf.split(X):
        train_data = X.iloc[train_idx, :], y.iloc[train_idx]
        valid_data = X.iloc[valid_idx, :], y.iloc[valid_idx]


        # Подаем trials для перебора
        model, y_pred = fit_algorithm(trial,
                                      train_data,
                                      valid_data,
                                      GaussianProcessRegressor,
                                      init_params,
                                      grid,
                                      fit_params)

        y_true = valid_data[1]
        scores.append(mean_squared_error(y_pred, y_true, squared=False))
        models.append(model)
        break

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [178]:
%%time
# optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize", study_name="Gaussian_Regressor")
study.optimize(objective_gaussian,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True,)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 56.3 s, sys: 4.81 s, total: 1min 1s
Wall time: 44.3 s


In [179]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 15.257996328031775, params {'alpha': 0.20022036388334133}


In [180]:
%%time

init_params = {'random_state': r_s,}

fit_params = { }

best_Gaussian_params = init_params | study.best_trial.params

res = best_fold_model(GaussianProcessRegressor, X, y, best_Gaussian_params, fit_params)

fold: 0, test_RMSE: 15.257996328031775
Best fold number = 0

fold: 1, test_RMSE: 14.933693271399324
Best fold number = 1

fold: 2, test_RMSE: 14.542728822989204
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по 3 фолдам test_RMSE_mean = 14.9115
CPU times: user 1.22 s, sys: 20.5 ms, total: 1.24 s
Wall time: 1.24 s


In [181]:
bst_mdls.update({'Gaussian_model': res[0]})
bst_fold_scores.update({'Gaussian_target': res[2]})
bst_fold_scores

preds = res[0].predict(X_sub)

df_sub['Gaussian_target'] = preds
df_sub.head()

,car_id,Catboost_target,LGBM_target,TabNet_target,KNN_target,Elastic_target,RF_target,Gaussian_target
0,P17494612l,46.076274,50.694532,34.337772,42.158554,48.730902,45.247314,41.884536
1,N-1530212S,33.787246,33.181376,33.622017,43.264945,43.930278,45.247314,43.470480
2,B-1154399t,33.687965,30.189062,32.035740,37.782325,38.445516,45.247314,37.711009
3,F12725233R,34.721955,36.430008,35.568596,43.918047,37.973594,45.247314,45.404781
4,l-1139189J,33.800808,30.863509,32.248013,47.391793,51.561601,45.247314,44.869010


## SVM regressor

In [182]:
def objective_svm(trial, return_models=False):
    init_params = {}

    grid = {
            'max_iter': trial.suggest_int('max_iter', 50, 500),
            'C': trial.suggest_float("C", 0.2, 1),
           }

    fit_params = {}

    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    scores, models = [], []


    for train_idx, valid_idx in kf.split(X):
        train_data = X.iloc[train_idx, :], y.iloc[train_idx]
        valid_data = X.iloc[valid_idx, :], y.iloc[valid_idx]


        # Подаем trials для перебора
        model, y_pred = fit_algorithm(trial,
                                      train_data,
                                      valid_data,
                                      NuSVR,
                                      init_params,
                                      grid,
                                      fit_params)

        y_true = valid_data[1]
        scores.append(mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
        models.append(model)
        break

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [183]:
%%time
# optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize", study_name="SVM_Regressor")
study.optimize(objective_svm,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True,)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 30.7 s, sys: 199 ms, total: 30.9 s
Wall time: 21 s


In [184]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 12.908793034884791, params {'max_iter': 496, 'C': 0.9938181140645257}


In [185]:
%%time

init_params = {}

fit_params = { }

best_SVM_params = init_params | study.best_trial.params

res = best_fold_model(NuSVR, X, y, best_SVM_params, fit_params)

fold: 0, test_RMSE: 12.908793034884791
Best fold number = 0

fold: 1, test_RMSE: 12.996394674949004
Best fold number = 0

fold: 2, test_RMSE: 12.71656175157353
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по 3 фолдам test_RMSE_mean = 12.8739
CPU times: user 592 ms, sys: 4.66 ms, total: 597 ms
Wall time: 602 ms


In [186]:
bst_mdls.update({'SVM_model': res[0]})
bst_fold_scores.update({'SVM_target': res[2]})
bst_fold_scores

preds = res[0].predict(X_sub)

df_sub['SVM_target'] = preds
df_sub.head()

,car_id,Catboost_target,LGBM_target,TabNet_target,KNN_target,Elastic_target,RF_target,Gaussian_target,SVM_target
0,P17494612l,46.076274,50.694532,34.337772,42.158554,48.730902,45.247314,41.884536,38.840318
1,N-1530212S,33.787246,33.181376,33.622017,43.264945,43.930278,45.247314,43.470480,39.563326
2,B-1154399t,33.687965,30.189062,32.035740,37.782325,38.445516,45.247314,37.711009,30.165772
3,F12725233R,34.721955,36.430008,35.568596,43.918047,37.973594,45.247314,45.404781,43.976434
4,l-1139189J,33.800808,30.863509,32.248013,47.391793,51.561601,45.247314,44.869010,41.213814


## MLP regressor

In [187]:
def objective_mlp(trial, return_models=False):
    init_params = {'random_state': r_s}

    grid = {'hidden_layer_sizes': trial.suggest_int('hidden_layer_sizes', 50, 200),
            'alpha': trial.suggest_int('alpha', 50, 500),
            'batch_size': trial.suggest_int('batch_size', 50, 200),
            'n_iter_no_change': trial.suggest_int('n_iter_no_change', 10, 50),
            'max_iter': trial.suggest_int('max_iter', 10, 500),
            'learning_rate': trial.suggest_categorical("learning_rate", ['constant', 'invscaling', 'adaptive'],),
            'beta_1': trial.suggest_float("beta_1", 0.5, 1),
            'beta_2': trial.suggest_float("beta_2", 0.5, 0.9),
           }

    fit_params = {}

    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    scores, models = [], []


    for train_idx, valid_idx in kf.split(X):
        train_data = X.iloc[train_idx, :], y.iloc[train_idx]
        valid_data = X.iloc[valid_idx, :], y.iloc[valid_idx]


        # Подаем trials для перебора
        model, y_pred = fit_algorithm(trial,
                                      train_data,
                                      valid_data,
                                      MLPRegressor,
                                      init_params,
                                      grid,
                                      fit_params)

        y_true = valid_data[1]
        scores.append(mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
        models.append(model)
        break

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [188]:
%%time
# optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize", study_name="MLP_Regressor")
study.optimize(objective_mlp,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True,)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 1min 34s, sys: 8.47 s, total: 1min 43s
Wall time: 1min 22s


In [189]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 16.935719236488858, params {'hidden_layer_sizes': 113, 'alpha': 50, 'batch_size': 143, 'n_iter_no_change': 44, 'max_iter': 469, 'learning_rate': 'invscaling', 'beta_1': 0.8204667664159928, 'beta_2': 0.6913676007972235}


In [190]:
%%time

init_params = {'random_state': r_s}

fit_params = { }

best_MLP_params = init_params | study.best_trial.params

res = best_fold_model(MLPRegressor, X, y, best_MLP_params, fit_params)

fold: 0, test_RMSE: 16.935719236488858
Best fold number = 0

fold: 1, test_RMSE: 16.669434082058554
Best fold number = 1

fold: 2, test_RMSE: 16.01513513711868
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по 3 фолдам test_RMSE_mean = 16.5401
CPU times: user 2.02 s, sys: 2.75 ms, total: 2.03 s
Wall time: 2.03 s


In [191]:
bst_mdls.update({'MLP_model': res[0]})
bst_fold_scores.update({'MLP_target': res[2]})
bst_fold_scores

preds = res[0].predict(X_sub)

df_sub['MLP_target'] = preds
df_sub.head()

,car_id,Catboost_target,LGBM_target,TabNet_target,KNN_target,Elastic_target,RF_target,Gaussian_target,SVM_target,MLP_target
0,P17494612l,46.076274,50.694532,34.337772,42.158554,48.730902,45.247314,41.884536,38.840318,47.535795
1,N-1530212S,33.787246,33.181376,33.622017,43.264945,43.930278,45.247314,43.470480,39.563326,45.279852
2,B-1154399t,33.687965,30.189062,32.035740,37.782325,38.445516,45.247314,37.711009,30.165772,43.161717
3,F12725233R,34.721955,36.430008,35.568596,43.918047,37.973594,45.247314,45.404781,43.976434,44.418174
4,l-1139189J,33.800808,30.863509,32.248013,47.391793,51.561601,45.247314,44.869010,41.213814,46.617224


## XGBoost

In [192]:
def objective_xgb(trial, return_models=False):

    pruning_callback = XGBoostPruningCallback(trial, 'validation_0-rmse')

    callbacks = [xgb.callback.LearningRateScheduler(lr_gen_simple(trial.suggest_float("lr_init", 0.0001, 0.5), trial.suggest_int('n_rounds', 20, 350))),
                #  xgb.callback.EarlyStopping(rounds=trial.suggest_int("early_stopping_rounds", 1000, 1500), metric_name="RMSE"),
                 pruning_callback]


    init_params = {'eval_metric':'rmse',
                  #  'booster': 'dart',
                   'random_state': r_s,
                   'n_jobs':-1,
                   'n_estimators': trial.suggest_int('n_rounds', 50, 300),
                   'objective': 'reg:squarederror',
                   }

    grid = {
            #  'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.01),
            "booster": trial.suggest_categorical("booster", ["gbtree",
                                                            #  "gblinear",
                                                             "dart"
                                                             ]),
            # 'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'max_leaves': trial.suggest_int('max_leaves', 2, 10),
            'lambda': trial.suggest_int("lambda", 2, 50),
            'max_bin': trial.suggest_int('max_bin', 50, 300),
            'alpha': trial.suggest_int('alpha', 2, 50),
            'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 0.95),
            'subsample': trial.suggest_float("subsample", 0.5, 0.95),
            'min_child_weight': trial.suggest_float("min_child_weight", 1, 10),
        }


    fit_params = {'model__verbose': 0,
                  # 'model__early_stopping_rounds': 20
                  }

    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=r_s)

    scores, models = [], []


    for train_idx, valid_idx in kf.split(X):
        train_data = X.iloc[train_idx, :], y.iloc[train_idx]
        valid_data = X.iloc[valid_idx, :], y.iloc[valid_idx]


        # Подаем trials для перебора
        model, y_pred = fit_algorithm(trial,
                                      train_data,
                                      valid_data,
                                      XGBRegressor,
                                      init_params,
                                      grid,
                                      fit_params,
                                      callbacks=callbacks)

        y_true = valid_data[1]
        scores.append(mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
        models.append(model)
        break

    result = np.mean(scores)

    if return_models:
        return result, models
    else:
        return result

In [193]:
%%time
# optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            direction="minimize", study_name="XGB_Regressor")
study.optimize(objective_xgb,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True,)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 9min 45s, sys: 35.6 s, total: 10min 20s
Wall time: 6min 7s


In [194]:
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

Best trial: score 11.63239574432373, params {'lr_init': 0.03222962637740156, 'n_rounds': 177, 'booster': 'gbtree', 'max_leaves': 7, 'lambda': 34, 'max_bin': 95, 'alpha': 2, 'colsample_bytree': 0.9246270837705568, 'subsample': 0.77603406840116, 'min_child_weight': 3.4219137655430485, 'max_depth': 8, 'eta': 1.1235437470169198e-06, 'gamma': 1.0011120658350499e-07, 'grow_policy': 'depthwise'}


In [195]:
best_XGB_params = study.best_trial.params

lr_init = best_XGB_params.pop('lr_init')
n_rounds = best_XGB_params.pop('n_rounds')


callbacks = [xgb.callback.LearningRateScheduler(lr_gen_simple(lr_init, n_rounds)),
            #  xgb.callback.EarlyStopping(rounds=early_stopping_rounds, #metric_name="RMSE")
            ]


init_params = {'eval_metric':'rmse',
                  #  'booster': 'dart',
                   'random_state': r_s,
                   'n_jobs':-1,
                   'n_estimators': n_rounds,
                   'objective': 'reg:squarederror',
                   }

fit_params = {'model__verbose': 0,
              # 'model__early_stopping_rounds': n_est
              }

best_XGB_params = init_params | best_XGB_params

# best_XGB_params

res = best_fold_model(XGBRegressor, X, y, best_XGB_params, fit_params, callbacks=callbacks)

fold: 0, test_RMSE: 11.63239574432373
Best fold number = 0

fold: 1, test_RMSE: 11.92117977142334
Best fold number = 0

fold: 2, test_RMSE: 11.535070419311523
Best fold number = 2

--------------------------------------------------------------------------------
Среднее значение метрики по 3 фолдам test_RMSE_mean = 11.696200370788574


In [196]:
bst_mdls.update({'XGB_model': res[0]})
bst_fold_scores.update({'XGB_target': res[2]})
bst_fold_scores

preds = res[0].predict(X_sub)

df_sub['XGB_target'] = preds
df_sub.head()

,car_id,Catboost_target,LGBM_target,TabNet_target,KNN_target,Elastic_target,RF_target,Gaussian_target,SVM_target,MLP_target,XGB_target
0,P17494612l,46.076274,50.694532,34.337772,42.158554,48.730902,45.247314,41.884536,38.840318,47.535795,46.839203
1,N-1530212S,33.787246,33.181376,33.622017,43.264945,43.930278,45.247314,43.470480,39.563326,45.279852,33.964905
2,B-1154399t,33.687965,30.189062,32.035740,37.782325,38.445516,45.247314,37.711009,30.165772,43.161717,30.308840
3,F12725233R,34.721955,36.430008,35.568596,43.918047,37.973594,45.247314,45.404781,43.976434,44.418174,37.957218
4,l-1139189J,33.800808,30.863509,32.248013,47.391793,51.561601,45.247314,44.869010,41.213814,46.617224,31.781811


In [197]:
bst_fold_scores

{'Catboost_target': 11.424122917098806,
 'LGBM_target': 11.566209261852029,
 'TabNet_target': 12.334204,
 'KNN_target': 14.701417459142696,
 'Elastic_target': 14.439893325533072,
 'RF_target': 17.108867104504995,
 'Gaussian_target': 14.542728822989204,
 'SVM_target': 12.71656175157353,
 'MLP_target': 16.01513513711868,
 'XGB_target': 11.53507}

In [118]:
# bst_mdls

# Блендинг

In [233]:
df_sub_c = df_sub[['car_id',
                 'Catboost_target',
                 'LGBM_target',
                #  'TabNet_target',
                #  'KNN_target',
                #  'Elastic_target',
                #  'RF_target',
                #  'Gaussian_target',
                #  'SVM_target',
                #  'MLP_target'
                 'XGB_target'
]]
df_sub_c.head()

,car_id,Catboost_target,LGBM_target,XGB_target
0,P17494612l,46.076274,50.694532,46.839203
1,N-1530212S,33.787246,33.181376,33.964905
2,B-1154399t,33.687965,30.189062,30.308840
3,F12725233R,34.721955,36.430008,37.957218
4,l-1139189J,33.800808,30.863509,31.781811


In [234]:
best_scores_chosen = {}
# for el in df_sub.columns:
#     if el in bst_fold_scores:
#       best_scores_chosen.update({el: bst_fold_scores[el]})

[best_scores_chosen.update({el: bst_fold_scores[el]}) for el in df_sub_c.columns if el in bst_fold_scores]
# best_scores_chosen

scores = 1 / np.array(list(best_scores_chosen.values()))
all_sum = np.sum(scores)
weights_ = scores / all_sum
# weights_

weights = {}
for f, b in zip(best_scores_chosen, weights_):
    weights.update({f: b})
print(weights)

{'Catboost_target': 0.33578488589053, 'LGBM_target': 0.331659900255269, 'XGB_target': 0.332555213854201}


In [ ]:
# for i in best_scores_chosen:
#   print(i)

In [235]:
def forward(row):
    average = 0
    for el in best_scores_chosen:
       average += row[el] * weights[el]

    return average

df_sub['target_reg'] = df_sub.apply(forward, axis=1)


# weights = [1]
# def forward(row):
#     average = 0
#     for i, el in enumerate(best_scores_chosen):
#         average += row[el] * weights[i]

#     return average

df_sub_c['target_reg'] = df_sub_c.apply(forward, axis=1)
df_sub_c.head()

,car_id,Catboost_target,LGBM_target,XGB_target,target_reg
0,P17494612l,46.076274,50.694532,46.839203,47.861681
1,N-1530212S,33.787246,33.181376,33.964905,33.645384
2,B-1154399t,33.687965,30.189062,30.308840,31.403773
3,F12725233R,34.721955,36.430008,37.957218,36.364351
4,l-1139189J,33.800808,30.863509,31.781811,32.155196


In [236]:
df_sub_csv = df_sub_c[['car_id', 'target_reg']]
df_sub_csv.to_csv('3_boosters.csv', index=False)

In [ ]:
catboost = 'RMSE вашего решения 11.9. Нужно не более 11.85'
lgbm = 'RMSE вашего решения 12.2. Нужно не более 11.85'
tabnet = 'RMSE вашего решения 13.2. Нужно не более 11.85'
knn = 'RMSE вашего решения 15.0. Нужно не более 11.85'
svm = 'RMSE вашего решения 13.2. Нужно не более 11.85'
xgb = 'RMSE вашего решения 12.2. Нужно не более 11.85'

In [ ]:
TabNet_model LGBM_model Catboost_model KNN_model Elastic_model RF_model Gaussian_model SVM_model MLP_model XGB_model

# Стекинг

In [ ]:
preparing_features(RandomForestRegressor, RandomForestRegressor)

In [345]:
# ctbrgr = CatBoostRegressor(subsample=0.95,
#                                     min_data_in_leaf=350,
#                                     max_bin=290,
#                                     learning_rate=0.06,
#                                     l2_leaf_reg=2,
#                                     depth=7,
#                            cat_features=cat_features
#                            )
# lgbmrgr = LGBMRegressor(subsample=0.95,
#                         num_leaves=9,
#                         min_data_in_leaf=8,
#                         max_depth=11,
#                         max_bin=60,
#                         lambda_l2=2,
#                         colsample_bytree=0.9
#     )
rfrgr = RandomForestRegressor(n_estimators=512,
                              max_depth=5,
                              max_features='sqrt',
                              random_state=r_s)

preprocessor, pipe = preparing_features(RandomForestRegressor, rfrgr)

In [353]:
models = [#[ctbrgr, lgbmrgr], # Level 1
          # [rfrgr]
          [pipe]
          ] # Level 2
# models = [[lgbmrgr], # Level 1
#           [rfrgr]] # Level 2

In [348]:
!git clone https://github.com/nedokormysh/my_pystacknet_corrected
# print(os.listdir("kaggle_pystacknet/pystacknet"))
# !pip install "/content/my_pystacknet_corrected/pystacknet"
!pip install "/content/my_pystacknet_corrected/pystacknet" -q
# import pystacknet

fatal: destination path 'my_pystacknet_corrected' already exists and is not an empty directory.
  Preparing metadata (setup.py) ... done


In [245]:
import joblib
import sys

sys.modules['sklearn.externals.joblib'] = joblib

from pystacknet.pystacknet import StackNetRegressor

In [354]:
# Specify parameters for stacked model and begin training
model = StackNetRegressor(models,
                           metric="rmse",
                           folds=4,
                           restacking=False,
                           use_retraining=True,
                          #  use_proba=True, # To use predict_proba after training
                           random_state=0,
                          #  n_jobs=-1,
                           verbose=1)

# Fit the entire model tree
# del models
model

StackNetRegressor(folds=4, metric=<function rmse at 0x79e52c799c60>,
                  models=[[Pipeline(steps=[('feature_preprocessor',
                                            ColumnTransformer(transformers=[('num',
                                                                             Pipeline(steps=[('imputer',
                                                                                              SimpleImputer(strategy='median')),
                                                                                             ('scaler',
                                                                                              StandardScaler())]),
                                                                             ['car_rating',
                                                                              'year_to_start',
                                                                              'riders',
                                                                              'year_to_work',
                                                                              'mean_rating',
                                                                              'distance_sum',
                                                                              'rating_min',
                                                                              'speed_max',
                                                                              'user_ride_quality_median',
                                                                              'user_uniq']),
                                                                            ('cat',
                                                                             Pipeline(steps=[('encoder',
                                                                                              OneHotEncoder(handle_unknown='ignore'))]),
                                                                             ['model',
                                                                              'car_type',
                                                                              'fuel_type'])])),
                                           ('model',
                                            RandomForestRegressor(max_depth=5,
                                                                  max_features='sqrt',
                                                                  n_estimators=512,
                                                                  random_state=7575))])]],
                  random_state=0, verbose=1)

In [355]:
model.fit(X_train, y_train)

====================== Start of Level 0 ======================
Input Dimensionality 13 at Level 0 
1 models included in Level 0 


ValueError: ignored

In [ ]:
X_train.head()

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,user_uniq
2131,10,0,0,1.651156,0.569181,0.298614,0.430868,-1.060147,0.106089,-0.237126,0.678064,0.103852,0.118988
2292,14,3,1,0.487187,0.569181,0.702344,0.430868,-1.536201,-0.355484,-0.237126,0.644183,-0.026325,-0.197727
22,14,3,1,0.267718,-0.617417,-0.701867,0.894040,-1.386850,-0.643211,-0.237126,0.621595,1.189970,-0.197727
1092,16,3,1,0.867338,-0.617417,-0.603727,-0.958651,-0.826787,0.407372,-0.237126,0.663006,0.310841,-0.197727
2336,22,1,1,-0.416162,0.569181,0.847479,-0.958651,0.377350,1.467838,1.531285,0.301606,-1.442524,0.435703


In [ ]:
# model.predict(X_sub)

preds = model.predict(X_sub.values)
preds = target_scaler.inverse_transform(np.array(preds).reshape(-1, 1))
preds[:5]

====================== Start of Level 0 ======================
1 estimators included in Level 0 
====================== Start of Level 1 ======================
1 estimators included in Level 1 


array([[42.37205794],
       [34.04430032],
       [33.09762789],
       [33.32164297],
       [33.17605187]])

In [ ]:
df_sub['target_reg'] = preds
# df_sub
# df_sub = df_sub[['car_id', 'target_reg']]
df_sub.to_csv('first_stacked.csv', index=False)

##Simple Stacking

In [279]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier, StackingRegressor
from sklearn.linear_model import LogisticRegression

In [ ]:
TabNet_model LGBM_model Catboost_model KNN_model Elastic_model RF_model Gaussian_model SVM_model MLP_model XGB_model

In [312]:
# список базовых моделей
estimators = [
    ("SVMoost", bst_mdls['SVM_model']),
    ("LightGBM", bst_mdls['LGBM_model']),
    ("CatBoost", bst_mdls['Catboost_model']),
]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingRegressor(
    estimators=estimators,
    # final_estimator=NuSVR(verbose=False),
    final_estimator=RandomForestRegressor(n_estimators = 10_000,
                                           max_depth = 5,
                                           verbose=False),
    n_jobs=-1,
    verbose=False,
)

stacking_regressor = meta_model
stacking_regressor.fit(X, y)

StackingRegressor(estimators=[('SVMoost',
                               TransformedTargetRegressor(regressor=Pipeline(steps=[('feature_preprocessor',
                                                                                     ColumnTransformer(transformers=[('num',
                                                                                                                      Pipeline(steps=[('imputer',
                                                                                                                                       SimpleImputer(strategy='median')),
                                                                                                                                      ('scaler',
                                                                                                                                       StandardScaler())]),
                                                                                                                      ['car_rating',
                                                                                                                       'year_to_start',
                                                                                                                       'riders',
                                                                                                                       'year_to_work',
                                                                                                                       'mean_rating',
                                                                                                                       'distance_sum',
                                                                                                                       'rating_min',
                                                                                                                       'speed...
                                                          transformer=StandardScaler())),
                              ('CatBoost',
                               TransformedTargetRegressor(regressor=Pipeline(steps=[('feature_preprocessor',
                                                                                     <__main__.CustomTransformer object at 0x79e54b83f640>),
                                                                                    ('model',
                                                                                     <catboost.core.CatBoostRegressor object at 0x79e53ad9b0d0>)]),
                                                          transformer=StandardScaler()))],
                  final_estimator=RandomForestRegressor(max_depth=5,
                                                        n_estimators=10000,
                                                        verbose=False),
                  n_jobs=-1, verbose=False)

In [330]:
preds = stacking_regressor.predict(X_sub)

In [309]:
df_sub_st = df_sub[['car_id',]]
df_sub_st['target_reg'] = preds
df_sub_st
df_sub_st.to_csv('stacked_3.csv', index=False)

# Ансамбль.

In [317]:
!git clone https://github.com/MLWave/Kaggle-Ensemble-Guide

Cloning into 'Kaggle-Ensemble-Guide'...
remote: Enumerating objects: 116, done.
remote: Total 116 (delta 0), reused 0 (delta 0), pack-reused 116
Receiving objects: 100% (116/116), 23.29 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [329]:
!python /content/Kaggle-Ensemble-Guide/src/kaggle_vote.py '/content/stacked_*.csv'  '/content/tset.csv'

parsing: /content/stacked_2.csv
parsing: /content/stacked_3.csv
parsing: /content/stacked_1.csv
parsing: /content/stacked_.csv
wrote to /content/tset.csv
